In [1]:
%%javascript
IPython.notebook.save_notebook()
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [2]:
# #exclude
# from ffwdpy import JupNb
# jupnb = JupNb()
# jupnb.write_nb2py(nb_name)

In [3]:
%%javascript

IPython.tab_as_tab_everywhere = function(use_tabs) {
	if (use_tabs === undefined) {
		use_tabs = true; 
	}

	// apply setting to all current CodeMirror instances
	IPython.notebook.get_cells().map(
		function(c) {  return c.code_mirror.options.indentWithTabs=use_tabs;  }
	);
	// make sure new CodeMirror instances created in the future also use this setting
	CodeMirror.defaults.indentWithTabs=use_tabs;
	
	};

IPython.tab_as_tab_everywhere()

<IPython.core.display.Javascript object>

In [4]:
# Import arguments
import argparse
import sys

# Arg parse function.
def parse_args():
	parser = argparse.ArgumentParser(description='')
	parser.add_argument("--index", help="Float Argument).", type=float, default=0)
	parser.add_argument("--gpus", help="GPU's to be used.",
						type=str, default="8")
	parser.add_argument('--set', help='String Argument',
						default = 'training', type=str)
	# parser.add_argument('--set', help='String Argument',
	# 					default = 'validation', type=str)

	
	args = parser.parse_args()
	return args

# Check shell
try:
	shell = get_ipython().__class__.__name__
	sys.argv = sys.argv[0:1]
except:
	pass

# Parse the args.
args = parse_args()
args.index = args.index % 9

In [5]:
# GPU setup
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]=args.gpus

In [6]:
from torchvision import transforms
import torch
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import os
import numpy as np
import torch
import clip
from tqdm.notebook import tqdm
from torchvision import transforms
import matplotlib.pyplot as plt
import cv2
import random
import os
import numpy as np
from PIL import Image
import torchvision   
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import sklearn
from torch import nn
from torch.nn.utils.rnn import *
import time
cuda = torch.cuda.is_available()
print("cuda", cuda)
num_workers = 8 if cuda else 0
print(num_workers)
print("Torch version:", torch.__version__)

cuda True
8
Torch version: 1.7.1+cu110


# Load CLIP Model

In [7]:
print("Avaliable Models: ", clip.available_models())
model, preprocess = clip.load("RN50", jit=False) # clip.load("ViT-B/32") #

input_resolution = model.input_resolution #.item()
context_length = model.context_length #.item()
vocab_size = model.vocab_size #.item()

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)

Avaliable Models:  ['RN50', 'ViT-B/32']
Model parameters: 102,007,137
Input resolution: 224
Context length: 77
Vocab size: 49408


In [8]:
import sys
sys.path.insert(0, "/data3/puppala/projects/Vid_CLIP/")
from utils.labels import labels, sub_labels, val_labels

In [9]:
import utils

NameError: name 'utils' is not defined

In [ ]:
sys.path

In [ ]:
map_id = {}
i=0
for label in labels:
	map_id[label]=i
	i+=1


In [ ]:
import os
import pdb
ROOT = "/data3/puppala/data/kinetics_jpg/{}".format(args.set)
DEST = "/data3/puppala/data/kinetics_embeddings/{}".format(args.set)
print(ROOT)
print(DEST)
cnt = 1
for filename in labels:
	if filename not in os.listdir(ROOT):
		print(filename," - Missing")
		
if args.set == 'validation':
	labels = val_labels

In [ ]:
class_names = os.listdir(ROOT)
len_classes = len(labels)
split_var = np.ceil(len_classes/8)
num_samples_per_class = 100
print("Creating embeddings from {} - {}".format(args.index * split_var, args.index*split_var + split_var))
for cls_idx, cls_name in enumerate(labels):
	if cls_idx < args.index*split_var or cls_idx >= (args.index + 1)*split_var:
		continue
	class_file = os.path.join(ROOT,cls_name)
	random_tags = os.listdir(class_file)
	save_dir = os.path.join(DEST, cls_name)
	if not os.path.exists(save_dir):
		os.makedirs(save_dir)
	len_random_tags = len(random_tags)
	avg_time = []
	for rand_id, random_tag in enumerate(random_tags):
		if rand_id >= num_samples_per_class:
			continue
		try:
			start_time = time.time()
			video_name = os.path.join(class_file,random_tag)
			save_dir = os.path.join(DEST, cls_name)
			file_save = os.path.join(save_dir, random_tag + '.npz')
			if os.path.exists(file_save):
				continue
			count = 0
			frames = sorted([ x for x in os.listdir(video_name) if x.endswith('.jpg')])
			N = len(frames)
			if N >=100:
				n = N//100
			else:
				n = 1
			selected_frames = np.arange(0,N,n).tolist()[0:100]
			images = []
			for frame in frames:
				if count in selected_frames:
					tmp = str(count)
					image = Image.open(os.path.join(video_name,frame))
					image = preprocess(image)
# 					image = torch.unsqueeze(image, 0)
					image = image.cuda()
					images.append(image)
				count+=1

			images_batch = torch.stack(images, dim=0)
			image_features, preattention_features = model.encode_image(images_batch , feat=True)
# 			print(image_features.shape, preattention_features.shape)
			image_features /= (image_features.norm(dim=-1, keepdim=True)  + 10e-8)
			image_features = image_features.detach().cpu().numpy()

			label = map_id[cls_name]*np.ones(image_features.shape[0])
			save_dir = os.path.join(DEST, cls_name)
			file_save = os.path.join(save_dir, random_tag)
			np.savez(file_save, data=image_features,label=label)
			# np.savez(file_save + '_preattention_features', data=preattention_features.detach().cpu().numpy(), label=label)
			end_time = time.time()
			avg_time.append(end_time - start_time)
			print(cls_idx - args.index*split_var, '/', np.ceil(len_classes/8), '-', rand_id, '/', 
				  len_random_tags, '\t time taken - ', np.round(np.mean(avg_time), 3), end='\r')
		except Exception as e:
			print(e)
